# First candidate score

### We have weighed the requirements as follows:

In a radius of 500m around the potential location:
 - kindergarten: 10 points / each
 - vegan restaurants: 8 points / each
 - Starbucks: 5 points / each (at least 1 Starbuck required)
 - bars: 2 points / each

### First, we must build a database with venues 800m around the candidate:

In [1]:
import pandas as pd
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE

In [2]:
from src import api_functions as af
from src import cleaning_functions as cf

### let's bring our first candidate...

In [79]:
df = pd.read_csv("DATA/candidates.csv")
place=df.iloc[0]                                      # change this for another place
place

Unnamed: 0                0
city          San Francisco
state                    CA
latitude           37.76785
longitude       -122.392861
Name: 0, dtype: object

### ... and build a database of venues around it:

In [5]:
location = f"{place.latitude},{place.longitude}"
venues=[]
radio=800

In [8]:
requi=["vegan","kindergarten","Starbucks","bar"]

In [9]:
for req in requi:
    resp=af.google(req,radio,location)
    clean = cf.limpiame(resp,req)
    for element in clean:
        venues.append(element)

In [13]:
name="place0.json"                                      # change this for another place

In [15]:
with open(f"DATA/{name}", 'w') as f:
    json.dump(venues, f)

### let's move to mongoDB...

In [16]:
client = MongoClient("localhost:27017")
db = client.get_database("FindLoc")

In [88]:
places = db.get_collection("place0")                     # change this for another place

In [31]:
db.place0.create_index([("location", GEOSPHERE)])        # change this for another place

'location_2dsphere'

### ... and make some queries:

In [37]:
locationfloat=[float(e) for e in location.split(",")]
locationreverse=[locationfloat[1],locationfloat[0]]
coord={"type":"Point", "coordinates": locationreverse}
proj = {"_id":0,"tipo":1,"nombre":1}
meters=500

In [54]:
requi

['vegan', 'kindergarten', 'Starbucks', 'bar']

In [80]:
weights=[10,8,5,2]

In [61]:
score=[]
for req in requi:
    cond = {"tipo":req}
    query = {"location": {"$near": {"$geometry": coord, "$maxDistance": meters}}}
    lis = list(places.find({"$and":[cond,query]},proj))
    score.append(len(lis))

In [62]:
score

[1, 0, 2, 0]

### For a total score of:

In [74]:
if score[2]==0:
    print ("At least 1 Starbucks is mandatory!")
else:
    total_score = sum([a*b for a,b in zip(score,weights)])

In [75]:
total_score

20

## looks like a great place!

## Bonus: mapping the venues

In [81]:
import folium

In [102]:
mymap = folium.Map(location=locationfloat, zoom_start=15, tiles="cartodbpositron")

In [117]:
icono = folium.Icon(color="red",icon_color="white",)
candi = folium.Marker(location=[place.latitude,place.longitude], tooltip="candidate",icon=icono)
candi.add_to(mymap);

In [123]:
rad = folium.Circle(location=[place.latitude,place.longitude],radius=500,color="green")
rad.add_to(mymap);

In [97]:
query = {"location": {"$near": {"$geometry": coord, "$maxDistance": meters}}}
proj = {"_id":0,"tipo":1,"nombre":1,"location.coordinates":1}
sitios = list(places.find(query,proj))

In [110]:
for element in sitios:
    mark = folium.Marker(location=[element["location"]["coordinates"][-1],element["location"]["coordinates"][0]], tooltip=element["nombre"])
    mark.add_to(mymap)

In [124]:
mymap